In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot
from math import sqrt
from pytz import timezone

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, Normalizer, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Conv1D
from keras import optimizers
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [3]:
import dovahkiin as dk
from dovahkiin.strategy import StrategyBase
from dovahkiin.feature.Amibroker import *
from dovahkiin.feature.StrategyUtility import *
from dovahkiin.OptimizeParam import OptimizeParam


class CrossOver(StrategyBase):

    """
    样本策略，均线交叉
    """

    params = {}
    params["stop_ratio"] = OptimizeParam("stop", 9.7, 0.1, 12, 0.1)
    params["short_ratio"] = OptimizeParam("shortPeriod", 0.19, 0.1, 0.5, 0.01)
    params["longPeriod"] = OptimizeParam("longPeriod", 24, 20, 60, 1)
    params["threshold_multiplier"] = OptimizeParam("threshold_multiplier", 1.1, 0.1, 4, 0.1)
    params["linreg_lookback"] = OptimizeParam("linreg_lookback", 46, 10, 60, 1)
    params["linreg_slope_coeff"] = OptimizeParam("slope coeff", 0.15, 0.05, 0.5, 0.05)
    params["cond3_coeff"] = OptimizeParam("cond3_coeff", 1.75, 1, 4, 0.25)


    def __init__(self, dataframe, params=None):
        super().__init__(dataframe, params)

    def strategy(self):

        """
        策略的逻辑
        """

        recentATR = ATR(self.C, self.H, self.L, 100, False)
        threshold = self.optimize("threshold_multiplier") * recentATR
        linreg_slope_coeff = self.optimize("linreg_slope_coeff")
        linreg_lookback = int(self.optimize("linreg_lookback"))
        long_period = int(self.optimize("longPeriod"))

        short_period = int(self.optimize("short_ratio") * long_period)
        short_line = MA(self.C, short_period)
        long_line = MA(self.C, self.optimize("longPeriod"))
        
        print("short period", short_period)
        print("linreg lookback", linreg_lookback)
        print("long period", long_period)

        close_slope = LinRegSlope(self.C, short_period)
        short_slope = LinRegSlope(short_line, linreg_lookback)

        # Long logic
        bcond1_1 = (self.C > long_line) & (self.C > short_line)
        bcond1_2 = long_line < short_line
        bcond1_3 = abs(short_line - long_line) > threshold
        bcond1 = bcond1_1 & bcond1_2 & bcond1_3
        bcond2 = LinRegSlope(self.C, short_period) > linreg_slope_coeff * self.optimize("cond3_coeff") * recentATR
        bcond3 = short_slope > linreg_slope_coeff * recentATR
        BSIG = bcond1 & bcond2 & bcond3

        # Short logic
        scond1_1 = (self.C < long_line) & (self.C < short_line)
        scond1_2 = long_line > short_line
        scond1_3 = abs(short_line - long_line) > threshold
        scond1 = scond1_1 & scond1_2 & scond1_3
        scond2 = LinRegSlope(self.C, short_period) < (-1) * linreg_slope_coeff * self.optimize("cond3_coeff") * recentATR
        scond3 = short_slope < (-1) * linreg_slope_coeff * recentATR
        SSIG = scond1 & scond2 & scond3

        self.BUY = BSIG
        self.SHORT = SSIG
        
        return BSIG.values

In [4]:
dp = dk.DataParser()
dataframe = dp.get_data("cu")

In [5]:
strategy = CrossOver(dataframe)

In [6]:
strategy.strategy();

short period 4
linreg lookback 46
long period 24


In [7]:
sigs = strategy.BUY.astype(int) - strategy.SHORT.astype(int)
y = sigs.values

In [8]:
dataframe["long_line"] = MA(strategy.C, 24).values
dataframe["short_line"] = MA(strategy.C, int(24*0.19)).values
dataframe["ATR"] = ATR(strategy.C, strategy.H, strategy.L, 100, False).values
dataframe["threshold"] = 1.1 * dataframe["ATR"]
dataframe["short_close_slope"] = LinRegSlope(strategy.C, int(24*0.19)).values
dataframe["slope_threshold"] = 0.15 * 1.75 * dataframe["ATR"]
dataframe["short_slope"] = LinRegSlope(strategy.C, 46).values

In [9]:
del dataframe["open"]
del dataframe["high"]
del dataframe["low"]
del dataframe["volume"]
del dataframe["openint"]

In [10]:
X = dataframe.values

In [11]:
# get rid of NAN
X = X[200:]
y = y[200:]

In [12]:
assert(not np.isnan(X).any())
assert(not np.isnan(y).any())

In [13]:
X.shape

(958435, 8)

In [14]:
y.shape

(958435,)

In [15]:
def FullyConnected_Model():
    model = Sequential()
    model.add(Dense(128, input_shape=(8,), activation="relu"))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation="softmax"))
    return model

In [16]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

In [17]:
X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.3)

In [18]:
y_train_category = to_categorical(y_train, num_classes=3)

In [27]:
adam = optimizers.adam(lr=1e-19)
sgd = optimizers.SGD(lr=1e-8, decay=1e-9, momentum=0.9, nesterov=True, clipnorm=1.)
model = FullyConnected_Model()
# model.compile(loss="mse", optimizer=adam, metrics=[ 'mse'])
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=[ 'accuracy'])

In [28]:
batch_size = 1000

In [29]:
def trainFullyConnected_network():   
    num_epochs = 1
    for i in range(num_epochs):
        model.fit(
            np.nan_to_num(X_train),
            np.nan_to_num(y_train_category),
            epochs=100,
            batch_size=batch_size,
            verbose=1,
            shuffle=False,
            validation_split=0.2
        )
        
    return model

In [30]:
model = trainFullyConnected_network()

Train on 536724 samples, validate on 134181 samples
Epoch 1/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 2/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 3/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 4/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 5/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 6/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 7/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 8/100
536724/536724 [===========

536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 64/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 65/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 66/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 67/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 68/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 69/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - val_loss: 0.6275 - val_acc: 0.9611
Epoch 70/100
536724/536724 [==============================] - 2s - loss: 1.0450 - acc: 0.9352 - 

In [31]:
model.evaluate(X_test, to_categorical(y_test, num_classes=3))

285536/287530 [============================>.] - ETA: 0s

[0.63551933345506784, 0.96057107084478144]

In [28]:
y_predicated

array([[-0.01228218],
       [-0.11140327],
       [ 0.0215313 ],
       ..., 
       [ 0.08549614],
       [-0.02162056],
       [-0.18421821]], dtype=float32)

In [29]:
y_test

date_time
2014-12-03 23:35:00+08:00    0
2014-12-03 23:36:00+08:00    0
2014-12-03 23:37:00+08:00    0
2014-12-03 23:38:00+08:00    0
2014-12-03 23:39:00+08:00    0
2014-12-03 23:40:00+08:00    0
2014-12-03 23:41:00+08:00    0
2014-12-03 23:42:00+08:00    0
2014-12-03 23:43:00+08:00    0
2014-12-03 23:44:00+08:00    0
2014-12-03 23:45:00+08:00    0
2014-12-03 23:46:00+08:00    0
2014-12-03 23:47:00+08:00    0
2014-12-03 23:48:00+08:00    0
2014-12-03 23:49:00+08:00    0
2014-12-03 23:50:00+08:00    0
2014-12-03 23:51:00+08:00    0
2014-12-03 23:52:00+08:00    0
2014-12-03 23:53:00+08:00    0
2014-12-03 23:54:00+08:00    0
2014-12-03 23:55:00+08:00    0
2014-12-03 23:56:00+08:00    0
2014-12-03 23:57:00+08:00    0
2014-12-03 23:58:00+08:00    0
2014-12-03 23:59:00+08:00    0
2014-12-04 00:00:00+08:00    0
2014-12-04 00:01:00+08:00    0
2014-12-04 00:02:00+08:00    0
2014-12-04 00:03:00+08:00    0
2014-12-04 00:04:00+08:00    0
                            ..
2017-07-04 14:30:00+08:00    